In [30]:
# Importing Necessary Library
import requests
from bs4 import BeautifulSoup
import csv


In [31]:
# I have chosen Spencers as it was easy to get store list and its URL
url = 'http://www.spencersretail.com/store-list?page=2'

In [32]:
# Using request which allows us to send HTTP reequest and then store in response.
response = requests.get(url)

In [33]:
# Here parsing the HTML content of the response using BeautifulSoup and then storing them into bsoup
bsoup = BeautifulSoup(response.text, 'html.parser')

In [34]:
### Finding all the store names and their respective information
store_name = bsoup.find_all("span", class_="fn")
store_address = bsoup.find_all("span", itemprop="streetAddress")
postal_code = bsoup.find_all("span", class_="postal-code")
locality = bsoup.find_all("span", class_="locality")
region = bsoup.find_all("span", class_="region")
Contact = bsoup.find_all("span", itemprop="telephone")
Time = bsoup.find_all("div", class_="field-content")



In [35]:
# Creating a List where I can store full address of the store...
full_address=[]
for i in range(len(store_name)):
    full_address.append({
        store_address[i].text.strip() + postal_code[i].text.strip() + locality[i].text.strip() + region[i].text.strip()       
    })


In [36]:
# Here trying to get the Timing i.e the opening and closing time of the store..
Opening_at=[]
for i in range(1,len(Time),3):
    Opening_at.append(Time[i])
    
Closing_at = []
for i in range(2,len(Time),3):
    Closing_at.append(Time[i])


In [37]:
# Set up the API endpoint URL and headers to access the TrueWay Geocoding API
url = "https://trueway-geocoding.p.rapidapi.com/Geocode"

headers = {
	"X-RapidAPI-Key": "b2dcd3ea06msh60d0300569d7b6dp1f52a6jsn31f7da53e46b",
	"X-RapidAPI-Host": "trueway-geocoding.p.rapidapi.com"
}




In [38]:
# Using TrueWay Geocoding API to get the latitude and longitude of each store and store them.....
lati_long = []
for i in range(len(full_address)):
    querystring = {"address":full_address[i],"language":"en"}
    response = requests.request("GET", url, headers=headers, params=querystring)
    lati_long.append((response.json()['results'][0]["location"]))
                  

In [42]:
# Crating a Dictionary to store all the information.....
info = []
for i in range(len(store_name)):
    info.append({
        "Store Name": store_name[i].text.strip(),
        "Store Address": store_address[i].text.strip(),
        "Postal Code": postal_code[i].text.strip(),
        "Locality": locality[i].text.strip(),
        "Region": region[i].text.strip(),
        "Contact": Contact[i].text.strip(),
        "Opening At": Opening_at[i].text.strip(),
        "Closing At": Closing_at[i].text.strip(),
        "Lati-Long": lati_long[i]
    })

In [44]:
# Write the data into a CSV file called 'lepton_assignment.csv'
with open('lepton_assignment.csv', mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=info[0].keys())
    writer.writeheader()
    for row in info:
        writer.writerow(row)